# 01 - Canonical Dataset Build

This notebook creates the master district-week dataset by merging all three sources.

**Output**: `data/processed/master.parquet`

In [2]:
import pandas as pd
import numpy as np
import glob
import os

print("Libraries loaded.")

Libraries loaded.


## 1. Load Data

In [3]:
def load_dataset(pattern):
    files = glob.glob(os.path.join('../data/raw', pattern))
    print(f"Loading {len(files)} files for '{pattern}'")
    if not files:
        return pd.DataFrame()
    return pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

enrolment = load_dataset('*enrolment*.csv')
demo_update = load_dataset('*demographic*.csv')
bio_update = load_dataset('*biometric*.csv')

print(f"\nLoaded: Enrolment={len(enrolment):,}, Demo={len(demo_update):,}, Bio={len(bio_update):,}")

Loading 3 files for '*enrolment*.csv'
Loading 5 files for '*demographic*.csv'
Loading 4 files for '*biometric*.csv'

Loaded: Enrolment=1,006,029, Demo=2,071,700, Bio=1,861,108


In [4]:
# Parse dates
for df in [enrolment, demo_update, bio_update]:
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')

# Extract year and week
for df in [enrolment, demo_update, bio_update]:
    df['year'] = df['date'].dt.isocalendar().year.astype('Int64')
    df['week_number'] = df['date'].dt.isocalendar().week.astype('Int64')

print("Date parsing and week extraction complete.")

Date parsing and week extraction complete.


## 2. Identify Age Columns

In [5]:
# Find age columns dynamically
enrol_age_cols = [c for c in enrolment.columns if 'age' in c.lower()]
demo_age_cols = [c for c in demo_update.columns if 'age' in c.lower()]
bio_age_cols = [c for c in bio_update.columns if 'age' in c.lower()]

print(f"Enrolment age cols: {enrol_age_cols}")
print(f"Demographic age cols: {demo_age_cols}")
print(f"Biometric age cols: {bio_age_cols}")

# Identify child column (5-17)
enrol_child_col = [c for c in enrol_age_cols if '5' in c and '17' in c]
demo_child_col = [c for c in demo_age_cols if '5' in c and '17' in c]
bio_child_col = [c for c in bio_age_cols if '5' in c and '17' in c]

enrol_child_col = enrol_child_col[0] if enrol_child_col else None
demo_child_col = demo_child_col[0] if demo_child_col else None
bio_child_col = bio_child_col[0] if bio_child_col else None

print(f"\nChild columns: Enrol={enrol_child_col}, Demo={demo_child_col}, Bio={bio_child_col}")

Enrolment age cols: ['age_0_5', 'age_5_17', 'age_18_greater']
Demographic age cols: ['demo_age_5_17', 'demo_age_17_']
Biometric age cols: ['bio_age_5_17', 'bio_age_17_']

Child columns: Enrol=age_5_17, Demo=demo_age_5_17, Bio=bio_age_5_17


## 3. Aggregate to District-Week

In [6]:
# Define aggregation keys
keys = ['state', 'district', 'year', 'week_number']

# Get numeric columns for summing
enrol_num = enrolment.select_dtypes(include=[np.number]).columns.tolist()
demo_num = demo_update.select_dtypes(include=[np.number]).columns.tolist()
bio_num = bio_update.select_dtypes(include=[np.number]).columns.tolist()

# Remove year/week from aggregation columns
enrol_num = [c for c in enrol_num if c not in ['year', 'week_number']]
demo_num = [c for c in demo_num if c not in ['year', 'week_number']]
bio_num = [c for c in bio_num if c not in ['year', 'week_number']]

# Aggregate
enrol_agg = enrolment.groupby(keys, as_index=False)[enrol_num].sum()
demo_agg = demo_update.groupby(keys, as_index=False)[demo_num].sum()
bio_agg = bio_update.groupby(keys, as_index=False)[bio_num].sum()

print(f"Aggregated: Enrol={len(enrol_agg):,}, Demo={len(demo_agg):,}, Bio={len(bio_agg):,}")

Aggregated: Enrol=15,195, Demo=16,932, Bio=19,171


In [7]:
# Rename columns to avoid conflicts
enrol_agg['enroll_total'] = enrol_agg[enrol_num].sum(axis=1)
if enrol_child_col:
    enrol_agg['enroll_child'] = enrol_agg[enrol_child_col]
else:
    enrol_agg['enroll_child'] = 0
enrol_final = enrol_agg[keys + ['enroll_child', 'enroll_total']]

demo_agg['demo_update_total'] = demo_agg[demo_num].sum(axis=1)
if demo_child_col:
    demo_agg['demo_update_child'] = demo_agg[demo_child_col]
else:
    demo_agg['demo_update_child'] = 0
demo_final = demo_agg[keys + ['demo_update_child', 'demo_update_total']]

bio_agg['bio_update_total'] = bio_agg[bio_num].sum(axis=1)
if bio_child_col:
    bio_agg['bio_update_child'] = bio_agg[bio_child_col]
else:
    bio_agg['bio_update_child'] = 0
bio_final = bio_agg[keys + ['bio_update_child', 'bio_update_total']]

print("Columns selected for merge.")

Columns selected for merge.


## 4. Merge Datasets

In [8]:
# Outer merge all three
master = enrol_final.merge(demo_final, on=keys, how='outer')
master = master.merge(bio_final, on=keys, how='outer')

# Fill NaN with 0 for count columns
count_cols = ['enroll_child', 'enroll_total', 'demo_update_child', 'demo_update_total', 'bio_update_child', 'bio_update_total']
master[count_cols] = master[count_cols].fillna(0).astype(int)

print(f"Master dataset: {len(master):,} rows")
print(f"Columns: {master.columns.tolist()}")

Master dataset: 21,629 rows
Columns: ['state', 'district', 'year', 'week_number', 'enroll_child', 'enroll_total', 'demo_update_child', 'demo_update_total', 'bio_update_child', 'bio_update_total']


## 5. Compute Derived Metrics

In [9]:
# Bio/Demo ratio for children
master['bio_demo_ratio_child'] = master['bio_update_child'] / master['demo_update_child'].replace(0, np.nan)

# Gap metric
master['bio_demo_gap_child'] = master['demo_update_child'] - master['bio_update_child']

# Sort for lag calculation
master = master.sort_values(['state', 'district', 'year', 'week_number']).reset_index(drop=True)

# Lag features (previous week within same district)
master['lag1_bio_update_child'] = master.groupby(['state', 'district'])['bio_update_child'].shift(1)
master['lag1_bio_demo_ratio'] = master.groupby(['state', 'district'])['bio_demo_ratio_child'].shift(1)

print("Derived metrics computed.")
master.head()

Derived metrics computed.


,state,district,year,week_number,enroll_child,enroll_total,demo_update_child,demo_update_total,bio_update_child,bio_update_total,bio_demo_ratio_child,bio_demo_gap_child,lag1_bio_update_child,lag1_bio_demo_ratio
0,100000,100000,2025,36,0,200004,0,0,0,0,NaN,0,NaN,NaN
1,100000,100000,2025,37,0,400006,0,0,0,0,NaN,0,0.0,NaN
2,100000,100000,2025,38,0,200002,0,0,0,0,NaN,0,0.0,NaN
3,100000,100000,2025,43,1,100001,0,0,0,0,NaN,0,0.0,NaN
4,100000,100000,2025,46,0,200005,0,0,0,0,NaN,0,0.0,NaN


## 6. Save to Parquet

In [10]:
# Ensure output directory exists
os.makedirs('../data/processed', exist_ok=True)

# Save
output_path = '../data/processed/master.parquet'
master.to_parquet(output_path, index=False)

print(f"Saved to {output_path}")
print(f"Final shape: {master.shape}")
print(f"Columns: {master.columns.tolist()}")

Saved to ../data/processed/master.parquet
Final shape: (21629, 14)
Columns: ['state', 'district', 'year', 'week_number', 'enroll_child', 'enroll_total', 'demo_update_child', 'demo_update_total', 'bio_update_child', 'bio_update_total', 'bio_demo_ratio_child', 'bio_demo_gap_child', 'lag1_bio_update_child', 'lag1_bio_demo_ratio']


In [11]:
# Validation
print("\n=== Validation ===")
print(f"Null counts:\n{master.isnull().sum()}")
print(f"\nSample data:")
master.sample(5)


=== Validation ===
Null counts:
state                       0
district                    0
year                        0
week_number                 0
enroll_child                0
enroll_total                0
demo_update_child           0
demo_update_total           0
bio_update_child            0
bio_update_total            0
bio_demo_ratio_child     6053
bio_demo_gap_child          0
lag1_bio_update_child    1132
lag1_bio_demo_ratio      6856
dtype: int64

Sample data:


,state,district,year,week_number,enroll_child,enroll_total,demo_update_child,demo_update_total,bio_update_child,bio_update_total,bio_demo_ratio_child,bio_demo_gap_child,lag1_bio_update_child,lag1_bio_demo_ratio
16212,Sikkim,West,2025,27,0,0,0,0,131,2211732,NaN,-131,132.0,NaN
7831,Jharkhand,Simdega,2025,45,32,34244109,87,42597328,456,45103244,5.241379,-369,291.0,5.596154
11672,Maharashtra,Solapur,2025,42,178,113226213,372,174813974,3175,155380965,8.534946,-2803,0.0,NaN
4582,Delhi,North Delhi,2025,51,19,1100506,127,2312052,651,2091977,5.125984,-524,910.0,3.321168
10037,Madhya Pradesh,Maihar,2025,22,0,0,0,0,10,485958,NaN,-10,21.0,NaN
